## Redis
### Redis 支持多种数据结构
- String: 字符串
- Hash: 散列
- List: 列表
- Set: 集合
- Sorted Set: 有序集合

### 演示场景
1. 使用Hash存储文章的（ID、标题）数据；
2. 使用String存储每个文章的访问次数，可以每次加1计数（或者存储文章ID的标签、作者附加属性都可以）;
3. 使用List存储每个用户的访问文章的历史，按顺序记录;
4. 使用Set存储访问网站的所有用户ID的集合;
5. 使用Sorted Set存储网站热榜，排序分数(访问次数)就是权重;

In [4]:
import utils.redisDb as db

### 获取链接

In [5]:
redis_conn = db.get_conn()

### 清除当前数据库内的所有数据

In [6]:
for key in redis_conn.keys():
    redis_conn.delete(key)

## 1. 给网站新增几篇文章

In [7]:
# 使用hash,类似map的形式，存储（Id、标题）数据
for idx in range(101,106):
    redis_conn.hset("articles", str(idx),f"this is {idx} article title")

## 2.给用户展示文章列表

In [8]:
# 展示所有的文章列表
for article_id,article_title in redis_conn.hgetall("articles").items():
    ## 分隔符
    print("#"*30)
    ## 默认返回 bytes 类型
    print(article_id,article_title)
    ## 转化为str类型
    print(article_id.decode("utf-8"),article_title.decode("utf-8"))

##############################
b'101' b'this is 101 article title'
101 this is 101 article title
##############################
b'102' b'this is 102 article title'
102 this is 102 article title
##############################
b'103' b'this is 103 article title'
103 this is 103 article title
##############################
b'104' b'this is 104 article title'
104 this is 104 article title
##############################
b'105' b'this is 105 article title'
105 this is 105 article title


In [9]:
## 展示单文章的标题
redis_conn.hget("articles","105")

b'this is 105 article title'

## 3. 用户访问文章则产生行为记录

In [10]:
def user_visit(uid,article_id):
    """ 产生了行为：uid 访问了 article_id """
    # 1. String 结构，文章的访问次数加1
    redis_conn.incr(f"article_counter_{article_id}")
    # 2. List 结构，记录uid的访问列表
    redis_conn.lpush(f"user_visit_{uid}",str(article_id))
    # 3. Set结构，记录uid的全站集合
    redis_conn.sadd(f"all_visit_uids",str(uid))
    # 4. SortedSet结构，文章的热度加1
    redis_conn.zincrby("article_hots", 1, str(article_id))

In [11]:
# 模拟3个用户的访问记录
user_visit("uid_01","101")
user_visit("uid_01","102")
user_visit("uid_01","103")

user_visit("uid_02","102")
user_visit("uid_02","103")
user_visit("uid_02","104")

user_visit("uid_03","103")
user_visit("uid_03","104")
user_visit("uid_03","105")

## 4. 查询文章的访问次数

In [12]:
redis_conn.get(f"article_counter_104")

b'2'

In [13]:
redis_conn.get(f"article_counter_105")

b'1'

## 5. 展示一个用户的访问历史

In [14]:
redis_conn.lrange("user_visit_uid_01", 0, -1)

[b'103', b'102', b'101']

In [15]:
redis_conn.lrange("user_visit_uid_03",0 ,-1)

[b'105', b'104', b'103']

## 6. 展示访问全站的用户集合

In [16]:
redis_conn.smembers("all_visit_uids")

{b'uid_01', b'uid_02', b'uid_03'}

## 7. 展示文章热榜以及热度

In [17]:
redis_conn.zrange("article_hots",0, -1, withscores=True, desc=True)

[(b'103', 3.0), (b'104', 2.0), (b'102', 2.0), (b'105', 1.0), (b'101', 1.0)]